In [1]:
using Ferrite, SparseArrays
using FerriteGmsh

# Define Geometry

In [2]:
R_cond = 19.1e-3;
R_ins  = 18.4e-3;
R_sh   = 1e-3;
R_jac  = 8e-3;

r_cond = R_cond;             # Conductor
r_ins  = r_cond + R_cond;    # Insulator
r_sh   = r_ins + R_sh;       # Sheath
r_jac  = r_sh + R_jac;       # Jacket

# Mesh density
mshd_cond = 0.5e-3; 
mshd_ins  = R_ins / 10;
mshd_sh   = R_sh / 5;
mshd_jac  = 2e-3;

In [3]:
#gmsh.finalize()
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.add("cable_geo")

geo = gmsh.model.geo;

## Points
geo.addPoint(0, 0, 0, mshd_cond, 1)
geo.addPoint(r_cond, 0, 0, mshd_cond, 2)
geo.addPoint(0, r_cond, 0, mshd_cond, 3)
geo.addPoint(r_ins, 0, 0, mshd_sh, 4)
geo.addPoint(0, r_ins, 0, mshd_sh, 5)
geo.addPoint(r_sh, 0, 0, mshd_sh, 6)
geo.addPoint(0, r_sh, 0, mshd_sh, 7)
geo.addPoint(r_jac, 0, 0, mshd_jac, 8)
geo.addPoint(0, r_jac, 0, mshd_jac, 9)

## Curves
geo.addCircleArc(2, 1, 3, 1)
geo.addCircleArc(4, 1, 5, 2)
geo.addCircleArc(6, 1, 7, 3)
geo.addCircleArc(8, 1, 9, 4)

geo.addLine(1, 2, 5)
geo.addLine(2, 4, 6)
geo.addLine(4, 6, 7)
geo.addLine(6, 8, 8)

geo.addLine(3, 1, 9)
geo.addLine(5, 3, 10)
geo.addLine(7, 5, 11)
geo.addLine(9, 7, 12)

## Surfaces
geo.addCurveLoop([5, 1, 9], 1)
geo.addCurveLoop([6, 2, 10, -1], 2)
geo.addCurveLoop([7, 3, 11, -2], 3)
geo.addCurveLoop([8, 4, 12, -3], 4)

geo.addPlaneSurface([1], 1)
geo.addPlaneSurface([2], 2)
geo.addPlaneSurface([3], 3)
geo.addPlaneSurface([4], 4)

## Define domains
geo.addPhysicalGroup(2, [1], 1) # Conductor
geo.addPhysicalGroup(2, [2], 2) # Dielectric
geo.addPhysicalGroup(2, [3], 3) # Sheath
geo.addPhysicalGroup(2, [4], 4) # Jacket
gmsh.model.setPhysicalName(2, 1, "Conductor")
gmsh.model.setPhysicalName(2, 2, "Dielectric")
gmsh.model.setPhysicalName(2, 3, "Sheath")
gmsh.model.setPhysicalName(2, 4, "Jacket")

geo.addPhysicalGroup(1, [4], 1) # Jacket boundary
gmsh.model.setPhysicalName(1, 1, "jacket")

# Generate mesh and save
gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)

gmsh.write("geo/cable_geo.msh")
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 10%] Meshing curve 2 (Circle)
Info    : [ 20%] Meshing curve 3 (Circle)
Info    : [ 30%] Meshing curve 4 (Circle)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 50%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.0359051s, CPU 0.046875s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : [ 80%] Meshing surface 4 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 1.14176s, CPU 1.375s)
Info    : 16928 nodes 34532 elements
Info    : Writing 'geo/cable_geo.msh'...
Info    : Done writing 'ge

# Ferrite

In [4]:
# Load the mesh using FerriteGmsh
grid = saved_file_to_grid("geo/cable_geo.msh");

Info    : Reading 'geo/cable_geo.msh'...
Info    : 25 entities
Info    : 16928 nodes
Info    : 33626 elements
Info    : Done reading 'geo/cable_geo.msh'


In [5]:
# Create finite element interpolation and quadrature rule
dim   = 2
order = 2;
ip_fe  = Lagrange{dim, RefTetrahedron, order}()
ip_geo = Lagrange{dim, RefTetrahedron, 1}()
qr = QuadratureRule{dim, RefTetrahedron}(2 * order)
cellvalues = CellScalarValues(qr, ip_fe, ip_geo);

qr2 = QuadratureRule{dim, RefTetrahedron}(2)
cellvalues2 = CellScalarValues(qr2, ip_geo);

# Create handler for the degrees of freedom, and define the scalar potential field
dh = DofHandler(grid);
push!(dh, :Az, 1, ip_fe)
close!(dh)

DofHandler
  Fields:
    :Az, interpolation: Lagrange{2, RefTetrahedron, 2}(), dim: 1
  Dofs per cell: 6
  Total dofs: 67443

In [6]:
# Define the boundary conditions using a constraint handler
ch = ConstraintHandler(dh);

dbc_0 = Dirichlet(
    :Az,
    getfaceset(grid, "jacket"),
    (x,t) -> 0;
)
add!(ch, dbc_0)

close!(ch)
update!(ch, 0.0); # Since the BCs do not depend on time, update them once at t = 0.0

In [7]:
# Element assembly: computes the elementary matrix contributions Ke and fe
function assemble_element!(Ke::Matrix, Me::Matrix, fe::Vector, cellvalues::CellScalarValues, Jz, nu, sigma, omega)
    n_basefuncs = getnbasefunctions(cellvalues)
    
    # Reset to 0
    fill!(Ke, 0)
    fill!(Me, 0)
    fill!(fe, 0)
    
    # Loop over quadrature points
    for q_point in 1:getnquadpoints(cellvalues)
        # Get the quadrature weight
        dΩ = getdetJdV(cellvalues, q_point)
        # Loop over test shape functions
        for i in 1:n_basefuncs
            v  = shape_value(cellvalues, q_point, i)
            ∇v = shape_gradient(cellvalues, q_point, i)
            
            # Add contribution to fe
            fe[i] += Jz * v * dΩ
            
            # Loop over trial shape functions
            for j in 1:n_basefuncs
                u  = shape_value(cellvalues, q_point, j)
                ∇u = shape_gradient(cellvalues, q_point, j)
                
                # Add contribution to Ke & Me
                Ke[i, j] += nu * (∇v ⋅ ∇u) * dΩ
                Me[i, j] += 1im * sigma * omega * (v ⋅ u) * dΩ
            end
        end
    end
    
    return Ke, fe
end

# Global assembly: computes global matrix K and f using element contributions
#  This is quite fast because of the use of a pre-allocated sparse matrix pattern.
function assemble_global(cellvalues::CellScalarValues, K::SparseMatrixCSC, dh::DofHandler)
    # Allocate the element stiffness matrix and element force vector
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(Complex{Float64}, n_basefuncs, n_basefuncs)
    Me = zeros(Complex{Float64}, n_basefuncs, n_basefuncs)
    fe = zeros(Complex{Float64}, n_basefuncs)
    
    # Allocate global force vector f
    f = zeros(Complex{Float64}, size(K, 1))
    
    # Create an assembler
    assembler = start_assemble(K, f)
    # Loop over all cells
    for cell in CellIterator(dh)
        # Reinitialize cellvalues for this cell
        reinit!(cellvalues, cell)
        cid = cellid(cell);
        
        # Compute element contribution
        assemble_element!(Ke, Me, fe, cellvalues, Jz[cid], nu[cid], sigma[cid], omega)
        Ae = Ke + Me;
        
        # Assemble Ke and fe into K and f
        assemble!(assembler, celldofs(cell), Ae, fe)
    end
    return K, f
end

assemble_global (generic function with 1 method)

In [8]:
function add_cconstraint_pattern(K::SparseMatrixCSC, dh::DofHandler, dom_name)
    N = size(K)
    K = hcat(K, spzeros(N[1]))
    K = vcat(K, spzeros(1, N[2] + 1))

    cells = getcellset(grid, dom_name);
    for c in cells
        dofs = celldofs(dh, c);
        for dof in dofs
            Ferrite.add_entry!(K, N[1] + 1, dof)
            Ferrite.add_entry!(K, dof, N[2] + 1)
        end
    end
    
    return K;
end

function add_cconstraint(K::SparseMatrixCSC, f::Vector, dh::DofHandler, dom_name, ncc, Icoil)
    cc_idx = dh.ndofs.x + ncc;
    
    n = getnbasefunctions(cellvalues)
    cell_dofs = zeros(Int, n)
    ccells = getcellset(grid, dom_name);
    for (cell_num, cell) in enumerate(CellIterator(dh))
        if(cell_num ∈ ccells)
            # Reinitialize cellvalues for this cell
            reinit!(cellvalues, cell)
            celldofs!(cell_dofs, dh, cell_num)

            for q_point in 1:getnquadpoints(cellvalues)
                dΩ = getdetJdV(cellvalues, q_point)

                K[cc_idx, cell_dofs[q_point]] += -1im * omega * sigma[cell_num] * dΩ;
                K[cell_dofs[q_point], cc_idx] += -4/(pi*r_cond*r_cond) * dΩ;
            end
        end
    end
    
    f[cc_idx] = Icoil / 4;
    K[cc_idx, cc_idx] = 1;
    
    return K, f;
end

add_cconstraint (generic function with 1 method)

In [9]:
# Define element properties
I = 1000;

Ncells = length(dh.grid.cells);
Jz = zeros(Float64, Ncells);

sigma = zeros(Ncells);
sigma[collect(getcellset(grid, "Conductor"))] .= 36.9e6;
sigma[collect(getcellset(grid, "Sheath"))] .= 59.6e6;

mu0 = 4e-7 * pi;
mur = ones(Ncells);
nu  = 1 ./ (mu0 * mur);

omega = 2*pi * 50;

# Create sparsity pattern from mesh data and assemble the linear system
K = create_sparsity_pattern(dh);
K = add_cconstraint_pattern(K, dh, "Conductor");

K = convert(SparseMatrixCSC{Complex{Float64}, Int64}, K)
K, f = assemble_global(cellvalues, K, dh);

K, f = add_cconstraint(K, f, dh, "Conductor", 1, I);

# Apply the boundary conditions
apply!(K, f, ch)

# Solve the linear system
u = K \ f;

# Post-processing

In [10]:
function compute_B(cellvalues::CellScalarValues{dim,T}, dh::DofHandler, a) where {dim,T}
    n = getnbasefunctions(cellvalues)
    cell_dofs = zeros(Int, n)
    nqp = getnquadpoints(cellvalues)

    # Allocate storage for the fluxes to store
    E = [Ferrite.Vec{2,Complex{T}}[] for _ in 1:getncells(dh.grid)]

    for (cell_num, cell) in enumerate(CellIterator(dh))
        E_cell = E[cell_num]
        celldofs!(cell_dofs, dh, cell_num)
        ae = a[cell_dofs]
        reinit!(cellvalues, cell)

        for E_point in 1:nqp
            E_qp = - function_gradient(cellvalues, E_point, ae)
            push!(E_cell, E_qp)
        end
    end
    return E
end

function compute_Jz(cellvalues::CellScalarValues{dim, T}, dh::DofHandler, a) where {dim, T}
    n = getnbasefunctions(cellvalues)
    cell_dofs = zeros(Int, n)
    nqp = getnquadpoints(cellvalues)

    # Allocate storage for the current density
    J   = zeros(Complex{T}, getncells(dh.grid));
    Ic  = 0;
    Rac = 0;
    
    for (cell_num, cell) in enumerate(CellIterator(dh))
        celldofs!(cell_dofs, dh, cell_num)
        ae = a[cell_dofs]
        reinit!(cellvalues, cell)
        reinit!(cellvalues2, cell)
        
        J[cell_num] += -1im * sigma[cell_num] * omega * sum(ae) / getnquadpoints(cellvalues);
        
        if(cell_num ∈ getcellset(grid, "Conductor"))
            J[cell_num] += 4 / (pi*r_cond*r_cond) * a[end];
            
            for q_point in 1:getnquadpoints(cellvalues)
                dΩ = getdetJdV(cellvalues, q_point)
                
                Ic  += J[cell_num] * dΩ;
                Rac += norm(J[cell_num])^2 / sigma[cell_num] * dΩ;
            end
        end
    end
    
    Ic = 4 * Ic;
    Rac = real(4 * Rac / Ic^2);
    
    return J, Ic, Rac
end

E_gp = compute_B(cellvalues, dh, u);
B = collect(mean(norm.(E_gp[i])) for i = 1:getncells(dh.grid));

J, Ic, Rac = compute_Jz(cellvalues, dh, u);

In [11]:
vtk_grid("images/hv_cable", dh) do vtk
    vtk_point_data(vtk, dh, norm.(u))
    vtk_cell_data(vtk, norm.(B), "B")
    vtk_cell_data(vtk, norm.(J), "Jz")
end

1-element Vector{String}:
 "images/hv_cable.vtu"

In [12]:
Rac

2.6758170469290028e-5

In [13]:
norm(Ic)

999.635735139496

# Visualize using Makie.jl

In [ ]:
using CairoMakie, LaTeXStrings
using GeometryBasics

CairoMakie.activate!(type = "svg")

In [ ]:
# Obtain a GeometryBasics.Mesh object suitable for plotting with Makie
function get_cell_mesh(cell_val)
    node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
    nNode = length(node_ids)

    eType, eTag, eConn = gmsh.model.mesh.getElements(2);
    nEl = length(eTag[1])
    
    points   = zeros(Point{2, Float64}, nEl * 3);        # Array of vertex coordinates (x,y)
    trif     = zeros(TriangleFace{Int}, nEl);  # Array of triangular faces (n1, n2, n3)
    node_val = zeros(Float64, nEl * 3);

    for e = 1:nEl
        n1idx = 3 * (e - 1) + 1; n1 = eConn[1][n1idx];
        n2idx = 3 * (e - 1) + 2; n2 = eConn[1][n2idx];
        n3idx = 3 * (e - 1) + 3; n3 = eConn[1][n3idx];

        points[n1idx] = Point{2}(node_coord[3*(n1-1) + 1], node_coord[3*(n1-1) + 2])
        points[n2idx] = Point{2}(node_coord[3*(n2-1) + 1], node_coord[3*(n2-1) + 2])
        points[n3idx] = Point{2}(node_coord[3*(n3-1) + 1], node_coord[3*(n3-1) + 2])
        
        node_val[n1idx] = cell_val[e];
        node_val[n2idx] = cell_val[e];
        node_val[n3idx] = cell_val[e];

        trif[e] = (3 * (e - 1) + 1, 3 * (e - 1) + 2, 3 * (e - 1) + 3);
    end

    msh = GeometryBasics.Mesh(points, trif);
    
    return msh, node_val
end

In [ ]:
gmsh.finalize()
gmsh.initialize()
gmsh.open("geo/cable_geo.msh")

msh, colors = get_cell_mesh(norm.(J) * 1e-6);

f, ax, pl = mesh(msh, color = colors, colormap = :bluesreds, shading = false)
ax.aspect = AxisAspect(1)
Colorbar(f[1,2], pl, label = L"Current density $J$")
save("images/hv_cable_J.png", f)
current_figure()

![Result: Current Density](images/hv_cable_J.png)

In [ ]:
msh, colors = get_cell_mesh(norm.(B) * 1e3);

f, ax, pl = mesh(msh, color = colors, colormap = :bluesreds, shading = false)
ax.aspect = AxisAspect(1)
Colorbar(f[1,2], pl, label = L"Flux Density $B$ [mT]")
save("images/hv_cable_B.png", f)
current_figure()

![Result: Magnetic Flux Density](images/hv_cable_b.png)